# Practica diez

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica se usará la librería de numpy, pandas, matplotlib, y sklearn.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [ ]:
data_train = pd.read_csv('CelebA-10K-train.csv')
data_test = pd.read_csv('CelebA-10K-test.csv')

data_train.shape, data_test.shape

## Ejercicio uno

In [ ]:
X_train = data_train.iloc[:, 2:]
y_train = data_train.iloc[:, 1]
X_test = data_test.iloc[:, 2:]
y_test = data_test.iloc[:, 1]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


classifs = [
	('Regresión Logística', LogisticRegression(random_state=42)),
	('Perceptón', Perceptron(random_state=42)),
	('SVM', SVC(random_state=42)),
	('KNN(5)', KNeighborsClassifier(n_neighbors=5)),
	('Centroide más cercano', NearestCentroid())
]

from collections import defaultdict

table = defaultdict(list)

fig, ax = plt.subplots(5, 1, figsize=(5, 25))

for i, (name, model) in enumerate(classifs):
	model.fit(X_train, y_train)
	cm = confusion_matrix(y_test, model.predict(X_test))
	ConfusionMatrixDisplay(cm, display_labels=['male', 'female']).plot(ax=ax[i])
	ax[i].set_title(name)

	accuracy = np.trace(cm) / np.sum(cm)
	recall = cm[1, 1] / np.sum(cm[1, :])
	precision = cm[1, 1] / np.sum(cm[:, 1])
	specificity = cm[0, 0] / np.sum(cm[0, :])
	f1_score = 2 * precision * recall / (precision + recall)
	table['model'].append(name)
	table['accuracy'].append(accuracy)
	table['recall'].append(recall)
	table['precision'].append(precision)
	table['specificity'].append(specificity)
	table['f1_score'].append(f1_score)

pd.DataFrame(table)

El clasificador SVM es el que mayor accuracy ofrece con un 94%, seguido de la regresión logística que nos ofrece un 91%. Respecto a la sensibilidad, el SVM vuelve a ser el mejor con mucha diferencia con un 95%. En cuanto a la precisión, gana el knn con 5 vecinos con casi un 95%, seguido del 92% del SVM. La especificidad la vuelve a ganar el knn con un 97%, otra vez seguido del 94% del SVM. El mayor f1 se lo lleva con mucha diferencia el SVM con un 93%, el siguiente más cercano es la regresión logística un un 89%.

## Ejercicio dos

In [ ]:
X_train = data_train.iloc[:,2:]
y_train = data_train.iloc[:,1]
X_test = data_test.iloc[:,2:]
y_test = data_test.iloc[:,1]

In [ ]:
import attrs
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from collections import defaultdict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

models = [
    ("Logistic Regression", LogisticRegression(random_state=42)),
    ("Perceptron", Perceptron(random_state=42)),
    ("SVC", SVC(random_state=42, gamma="auto")),
    ("Gaussian", GaussianNB()),
    ("Nearest Centroid", NearestCentroid()),
]

fig, ax = plt.subplots(figsize=(7, 7))
fig2, ax2 = plt.subplots(5, 1, figsize=(5, 25))

for i, (name, model) in enumerate(models):
    model.fit(X_train, y_train)
    p = model.predict(X_test)
    c = confusion_matrix(y_test, p)
    ConfusionMatrixDisplay(c, display_labels=["male", "female"]).plot(ax=ax2[i])
    ax2[i].set_title(name)

    tn, fp, fn, tp = c.ravel()
    
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    ax.scatter(fpr, tpr, label=name)
    ax.plot([0, 1], [0, 1], color="black", linestyle="--")

ax.set_xlabel("FPR")
ax.set_ylabel("TPR")
ax.legend()
fig.show()

Se puede observar con la regresión logística es la que mayor ratio de verdaderos positivos tiene, así como la que menor ratio de falsos positivos tiene. Por lo tanto, es la que mejor clasifica los datos. Por otro lado, la Gausiana es la que peor clasifica los datos, ya que tiene el menor ratio de verdaderos positivos y el segundo mayor ratio de falsos positivos, solo por detrás del Nearest Centroid, sin embargo el Nearest Centroid al menos tiene una tasa de verdaderos positivos prácticamente igual que el de la regresión logística.

## Ejercicio tres

In [ ]:
from numpy import linspace
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

classifs = [    ('Naive Bayes', GaussianNB()),    ('Regresión Logística', LogisticRegression()),    ('SVM', SVC(probability=True))]

fig, ax = plt.subplots(figsize=(10, 10))

for name, model in classifs:
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, probs)
    auc_score = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f"{name} (AUC = {auc_score:.2f})")

# Añadir la línea punteada que representa el clasificador aleatorio
ax.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Clasificador aleatorio')

# Ajustar los límites de los ejes x e y
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

# Agregar etiquetas y leyenda
ax.set_xlabel('Tasa de Falsos Positivos')
ax.set_ylabel('Tasa de Verdaderos Positivos')
ax.legend()
plt.show()